In [12]:
import pandas as pd
import numpy as np
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import random

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option("display.max_columns", None)


In [13]:
def simulate_central_scenario(seed=1234):
    np.random.seed(seed)
    random.seed(seed)

    # Policy data
    date_range = pd.date_range(start="2016-01-01", end="2017-12-31")
    policy_data = []

    for date in date_range:
        count = np.random.poisson(700)
        for i in range(count):
            policy = {
                "date_UW": date,
                "date_lapse": date + relativedelta(years=1),
                "pol_prefix": int(date.year * 10000 + date.month * 100 + date.day)
            }
            policy_data.append(policy)

    dt_policy = pd.DataFrame(policy_data)
    dt_policy["pol_seq"] = dt_policy.groupby("pol_prefix").cumcount() + 1
    dt_policy["pol_number"] = (dt_policy["pol_prefix"] * 10000 + dt_policy["pol_seq"]).astype(str)

    # Coverage assignment
    grouped = dt_policy.groupby("date_UW")["pol_number"].count().reset_index(name="policycount")
    grouped["Cover_B"] = (grouped["policycount"] * 0.25).round().astype(int)
    grouped["Cover_BO"] = (grouped["policycount"] * 0.45).round().astype(int)
    grouped["Cover_BOT"] = grouped["policycount"] - grouped["Cover_B"] - grouped["Cover_BO"]
    dt_policy = dt_policy.merge(grouped, on="date_UW", how="left")

    dt_policy["Cover"] = "BO"
    dt_policy["rank"] = dt_policy.groupby("date_UW").cumcount() + 1
    dt_policy.loc[dt_policy["rank"] <= dt_policy["Cover_BOT"], "Cover"] = "BOT"
    dt_policy.loc[dt_policy["rank"] <= dt_policy["Cover_B"], "Cover"] = "B"

    # Assign brand/model/price
    brands = [1]*9 + [2]*6 + [3]*3 + [4]*2
    dt_policy["Brand"] = np.resize(brands, len(dt_policy))
    base_prices = {1: 600, 2: 550, 3: 300, 4: 150}
    dt_policy["Base_Price"] = dt_policy["Brand"].map(base_prices)

    model_seq = [3]*10 + [2]*7 + [1]*2 + [0]*1
    model_mult = {3: 1.15**3, 2: 1.15**2, 1: 1.15**1, 0: 1.0}
    dt_policy["Model"] = np.resize(model_seq, len(dt_policy))
    dt_policy["Model_mult"] = dt_policy["Model"].map(model_mult)
    dt_policy["Price"] = np.ceil(dt_policy["Base_Price"] * dt_policy["Model_mult"]).astype(int)

    dt_policy = dt_policy[["pol_number", "date_UW", "date_lapse", "Cover", "Brand", "Model", "Price"]]

    # Claim data
    claim_indices_B = np.random.choice(dt_policy.index, size=int(0.15 * len(dt_policy)), replace=False)
    dt_claim = dt_policy.loc[claim_indices_B, ["pol_number"]].copy()
    dt_claim["claim_type"] = "B"
    dt_claim["claim_count"] = 1
    dt_claim["claim_sev"] = np.random.beta(2, 5, size=len(dt_claim))

    indices_O = dt_policy[dt_policy["Cover"].isin(["BO", "BOT"])].index
    claim_indices_O = np.random.choice(indices_O, size=int(0.05 * len(indices_O)), replace=False)
    dt_O = dt_policy.loc[claim_indices_O, ["pol_number"]].copy()
    dt_O["claim_type"] = "O"
    dt_O["claim_count"] = 1
    dt_O["claim_sev"] = np.random.beta(5, 3, size=len(dt_O))
    dt_claim = pd.concat([dt_claim, dt_O])

    for m in [0, 1, 2, 3]:
        subset = dt_policy[(dt_policy["Cover"] == "BOT") & (dt_policy["Model"] == m)]
        n = int(0.05 * (1 + m) * len(subset))
        claim_indices_T = np.random.choice(subset.index, size=n, replace=False)
        dt_T = dt_policy.loc[claim_indices_T, ["pol_number"]].copy()
        dt_T["claim_type"] = "T"
        dt_T["claim_count"] = 1
        dt_T["claim_sev"] = np.random.beta(5, 0.5, size=len(dt_T))
        dt_claim = pd.concat([dt_claim, dt_T])

    dt_claim = dt_claim.merge(dt_policy[["pol_number", "date_UW", "Price", "Brand"]], on="pol_number", how="left")
    dt_claim["date_lapse"] = dt_claim["date_UW"] + pd.DateOffset(years=1)
    dt_claim["expodays"] = (dt_claim["date_lapse"] - dt_claim["date_UW"]).dt.days
    dt_claim["occ_delay_days"] = (dt_claim["expodays"] * np.random.rand(len(dt_claim))).astype(int)
    dt_claim["delay_report"] = np.floor(365 * np.random.beta(.4, 10, size=len(dt_claim))).astype(int)
    dt_claim["delay_pay"] = np.floor(10 + 40 * np.random.beta(7, 7, size=len(dt_claim))).astype(int)

    dt_claim["date_occur"] = dt_claim["date_UW"] + pd.to_timedelta(dt_claim["occ_delay_days"], unit='D')
    dt_claim["date_report"] = dt_claim["date_occur"] + pd.to_timedelta(dt_claim["delay_report"], unit='D')
    dt_claim["date_pay"] = dt_claim["date_report"] + pd.to_timedelta(dt_claim["delay_pay"], unit='D')
    dt_claim["claim_cost"] = (dt_claim["Price"] * dt_claim["claim_sev"]).round().astype(int)

    dt_claim["clm_prefix"] = dt_claim["date_report"].dt.strftime("%Y%m%d").astype(int)
    dt_claim["clm_seq"] = dt_claim.groupby("clm_prefix").cumcount() + 1
    dt_claim["clm_prefix"] = dt_claim["date_report"].dt.strftime("%Y%m%d")
    dt_claim["clm_seq"] = dt_claim.groupby("clm_prefix").cumcount() + 1

        # Use zfill for consistent formatting
    dt_claim["clm_number"] = dt_claim["clm_prefix"].astype(str) + dt_claim["clm_seq"].astype(str).str.zfill(4)


    # Keep only first claim per policy
    dt_claim["polclm_seq"] = dt_claim.groupby("pol_number").cumcount() + 1
    dt_claim = dt_claim[dt_claim["polclm_seq"] == 1]

    dt_claim = dt_claim[[
        "clm_number", "pol_number", "claim_type", "claim_count", "claim_sev",
        "date_occur", "date_report", "date_pay", "claim_cost"
    ]]

    return dt_policy.reset_index(drop=True), dt_claim.reset_index(drop=True)


In [14]:
dt_policy, dt_claim = simulate_central_scenario(1234)

print("Policy Sample:")
print(dt_policy.head())

print("\nClaim Sample:")
print(dt_claim.head())


Policy Sample:
     pol_number    date_UW date_lapse Cover  Brand  Model  Price
0  201601010001 2016-01-01 2017-01-01     B      1      3    913
1  201601010002 2016-01-01 2017-01-01     B      1      3    913
2  201601010003 2016-01-01 2017-01-01     B      1      3    913
3  201601010004 2016-01-01 2017-01-01     B      1      3    913
4  201601010005 2016-01-01 2017-01-01     B      1      3    913

Claim Sample:
     clm_number    pol_number claim_type  claim_count  claim_sev date_occur  \
0  201805090001  201707030353          B            1   0.639180 2018-04-24   
1  201701310001  201610100547          B            1   0.260456 2017-01-31   
2  201609250001  201604150398          B            1   0.070113 2016-09-15   
3  201705220001  201608220393          B            1   0.150239 2017-05-22   
4  201803240001  201706270312          B            1   0.371893 2018-03-19   

  date_report   date_pay  claim_cost  
0  2018-05-09 2018-06-11         254  
1  2017-01-31 2017-03-01   

In [15]:
# Merge claims with policy using policy number
dt_polclaim = pd.merge(dt_policy, dt_claim, on="pol_number", how="left")

# Fill missing values (i.e., policies with no claims)
long_future = pd.to_datetime("2199-12-31 23:59:59")
dt_polclaim["date_occur"] = dt_polclaim["date_occur"].fillna(long_future)
dt_polclaim["date_report"] = dt_polclaim["date_report"].fillna(long_future)
dt_polclaim["date_pay"] = dt_polclaim["date_pay"].fillna(long_future)
dt_polclaim["claim_count"] = dt_polclaim["claim_count"].fillna(0).astype(int)
dt_polclaim["claim_sev"] = dt_polclaim["claim_sev"].fillna(0)
dt_polclaim["claim_cost"] = dt_polclaim["claim_cost"].fillna(0).astype(int)

# Add exposure days
dt_polclaim["ExpoDays"] = ((dt_polclaim["date_lapse"] - dt_polclaim["date_UW"]).dt.days / 365).apply(np.ceil)
dt_polclaim = dt_polclaim[dt_polclaim["ExpoDays"] > 0]


In [16]:
from pandas.tseries.offsets import Day

# Generate time slices of 30 days
lst_date_slice = pd.date_range("2016-01-01", "2019-06-30", freq="30D")

# Add cumulative payment columns for each slice
for t in tqdm(lst_date_slice):
    col = f'P_t_{t.strftime("%Y%m%d")}'
    dt_polclaim[col] = np.where(dt_polclaim["date_pay"] <= t, dt_polclaim["claim_cost"], 0)


100%|█████████████████████████████████████████████████████████████████████████████████| 43/43 [00:00<00:00, 123.15it/s]


In [17]:
# View top rows
print("Preview of dt_policy:")
print(dt_policy.head())

# Save to CSV
#dt_policy.to_csv("dt_policy.csv", index=False)
#print("Saved dt_policy.csv")


Preview of dt_policy:
     pol_number    date_UW date_lapse Cover  Brand  Model  Price
0  201601010001 2016-01-01 2017-01-01     B      1      3    913
1  201601010002 2016-01-01 2017-01-01     B      1      3    913
2  201601010003 2016-01-01 2017-01-01     B      1      3    913
3  201601010004 2016-01-01 2017-01-01     B      1      3    913
4  201601010005 2016-01-01 2017-01-01     B      1      3    913


In [18]:
# View top rows
print("\nPreview of dt_claim:")
print(dt_claim.head())

# Save to CSV
#dt_claim.to_csv("dt_claim.csv", index=False)
#print("Saved dt_claim.csv")



Preview of dt_claim:
     clm_number    pol_number claim_type  claim_count  claim_sev date_occur  \
0  201805090001  201707030353          B            1   0.639180 2018-04-24   
1  201701310001  201610100547          B            1   0.260456 2017-01-31   
2  201609250001  201604150398          B            1   0.070113 2016-09-15   
3  201705220001  201608220393          B            1   0.150239 2017-05-22   
4  201803240001  201706270312          B            1   0.371893 2018-03-19   

  date_report   date_pay  claim_cost  
0  2018-05-09 2018-06-11         254  
1  2017-01-31 2017-03-01         190  
2  2016-09-25 2016-10-29          24  
3  2017-05-22 2017-06-22         109  
4  2018-03-24 2018-04-26         271  


In [19]:
# View top rows
print("\nPreview of dt_polclaim (joined + timesliced):")
print(dt_polclaim.iloc[:, :15].head())  # Show first 15 columns for readability

# Save full dataset to CSV
#dt_polclaim.to_csv("dt_polclaim.csv", index=False)
print("Saved dt_polclaim.csv")



Preview of dt_polclaim (joined + timesliced):
     pol_number    date_UW date_lapse Cover  Brand  Model  Price clm_number  \
0  201601010001 2016-01-01 2017-01-01     B      1      3    913        NaN   
1  201601010002 2016-01-01 2017-01-01     B      1      3    913        NaN   
2  201601010003 2016-01-01 2017-01-01     B      1      3    913        NaN   
3  201601010004 2016-01-01 2017-01-01     B      1      3    913        NaN   
4  201601010005 2016-01-01 2017-01-01     B      1      3    913        NaN   

  claim_type  claim_count  claim_sev          date_occur         date_report  \
0        NaN            0        0.0 2199-12-31 23:59:59 2199-12-31 23:59:59   
1        NaN            0        0.0 2199-12-31 23:59:59 2199-12-31 23:59:59   
2        NaN            0        0.0 2199-12-31 23:59:59 2199-12-31 23:59:59   
3        NaN            0        0.0 2199-12-31 23:59:59 2199-12-31 23:59:59   
4        NaN            0        0.0 2199-12-31 23:59:59 2199-12-31 23:59:59  